# Eval

- Metrike: Precision, Recall, F1, Accuracy (s TN) pri pragu t

- Rangirne metrike med pozitivnimi: Acc@1, P@5, MRR



In [1]:

# Parameters — prilagodi poti
PREDS_JSON = "notebook_straza/result/straza_match.json"
GOLD_JSON = "notebook_straza/gold/trbovlje_to_straza.json"
TOP_K = 5
THRESHOLD = 0.7
SCORE_FIELDS = ["combined_score", "score"]
RUN_TAG = "straza"  # uporabi za imena izvozov


In [2]:

# Imports & helpers
import sys
from importlib import reload
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

sys.path.append("notebook_brezice")

import json_eval_helpers as H
reload(H)

print("Na voljo helper funkcije:", [n for n in dir(H) if not n.startswith("_")])


Na voljo helper funkcije: ['Dict', 'List', 'Optional', 'Set', 'Tuple', 'compute_metrics_from_maps', 'compute_open_world_metrics', 'decide_label_with_threshold', 'json', 'load_gold_json', 'load_predictions_json', 'load_predictions_with_scores', 'normalize']


In [3]:

# Naloži gold in predikcije
gold_map = H.load_gold_json(GOLD_JSON)

pred_pairs_map = H.load_predictions_with_scores(
    PREDS_JSON,
    score_field_options=SCORE_FIELDS,
    top_k=TOP_K
)

len(gold_map), len(pred_pairs_map)


(7, 5)

In [4]:

# Open-world metrike (odlocitve + rangiranje med pozitivnimi)
open_world = H.compute_open_world_metrics(
    gold_map,
    pred_pairs_map,
    threshold=THRESHOLD,
    k=TOP_K,
    exclude_no_match_from_mrr=True
)
open_world


{'decision_metrics': {'threshold': 0.7,
  'TP': 5,
  'FP': 0,
  'FN': 0,
  'TN': 2,
  'precision': 1.0,
  'recall': 1.0,
  'f1': 1.0,
  'accuracy': 1.0},
 'ranking_metrics_among_positives': {'count_positives': 5,
  'acc_at_1': 1.0,
  'p_at_5': 0.2,
  'mrr': 1.0},
 'details': [{'source': 'Naziv',
   'gold': ['naziv'],
   'chosen': 'naziv',
   'outcome': 'TP',
   'candidates': ['naziv',
    'plačljivo',
    'cena/h',
    'lokacijska koordinata',
    'št. Park. Mest'],
   'scores': [0.9176725113391876,
    0.7553631859166281,
    0.6996665282682939,
    0.6835151878687051,
    0.669764114681043]},
  {'source': 'Lokacijska koordinata (GMAPS - WGS84)',
   'gold': ['lokacijska koordinata'],
   'chosen': 'lokacijska koordinata',
   'outcome': 'TP',
   'candidates': ['lokacijska koordinata',
    'št. Park. Mest',
    'cena/h',
    'naziv',
    'plačljivo'],
   'scores': [0.8633071862418076,
    0.676959463778664,
    0.6611295619676278,
    0.6537722783429282,
    0.6346841607404792]},
  {'sou

In [5]:

# Closed-world primerjava (Acc@1, P@5, MRR) — če helper to podpira
pred_map_labels = {s: [lab for (lab, _sc) in pairs] for s, pairs in pred_pairs_map.items()}
closed_metrics = H.compute_metrics_from_maps(gold_map, pred_map_labels, k=TOP_K)
closed_metrics


{'count': 5,
 'acc_at_1': 1.0,
 'p_at_5': 1.0,
 'mrr': 1.0,
 'details': [{'source_column': 'Naziv',
   'gold': ['cena/h',
    'lokacijska koordinata',
    'naziv',
    'plačljivo',
    'št. Park. Mest'],
   'predictions': {'naziv'},
   'hit': True,
   'rank_hit': 1,
   'hits_in_k': 1},
  {'source_column': 'Lokacijska koordinata (GMAPS - WGS84)',
   'gold': ['cena/h',
    'lokacijska koordinata',
    'naziv',
    'plačljivo',
    'št. Park. Mest'],
   'predictions': {'lokacijska koordinata'},
   'hit': True,
   'rank_hit': 1,
   'hits_in_k': 1},
  {'source_column': 'Plačljivo',
   'gold': ['cena/h',
    'lokacijska koordinata',
    'naziv',
    'plačljivo',
    'št. Park. Mest'],
   'predictions': {'plačljivo'},
   'hit': True,
   'rank_hit': 1,
   'hits_in_k': 1},
  {'source_column': 'Cena',
   'gold': ['cena/h',
    'lokacijska koordinata',
    'naziv',
    'plačljivo',
    'št. Park. Mest'],
   'predictions': {'cena/h'},
   'hit': True,
   'rank_hit': 1,
   'hits_in_k': 1},
  {'sourc

In [6]:

# Detajli po stolpcih + CSV izvoz
details_df = pd.DataFrame(open_world["details"])
display(details_df.head(20))

out_dir = Path("notebook_straza/results_eval/results_eval_json")
out_dir.mkdir(parents=True, exist_ok=True)
out_csv = out_dir / f"eval_json_details_{RUN_TAG}.csv"
details_df.to_csv(out_csv, index=False, encoding="utf-8")
print("Saved CSV:", out_csv)


,source,gold,chosen,outcome,candidates,scores
0,Naziv,[naziv],naziv,TP,"[naziv, plačljivo, cena/h, lokacijska koordina...","[0.9176725113391876, 0.7553631859166281, 0.699..."
1,Lokacijska koordinata (GMAPS - WGS84),[lokacijska koordinata],lokacijska koordinata,TP,"[lokacijska koordinata, št. Park. Mest, cena/h...","[0.8633071862418076, 0.676959463778664, 0.6611..."
2,Plačljivo,[plačljivo],plačljivo,TP,"[plačljivo, naziv, lokacijska koordinata, cena...","[0.9602607289950051, 0.7585886018616812, 0.700..."
3,Cena,[cena/h],cena/h,TP,"[cena/h, naziv, plačljivo, lokacijska koordina...","[0.8474521505832671, 0.773579337199529, 0.7018..."
4,Število parkirnih mest,[št. Park. Mest],št. Park. Mest,TP,"[št. Park. Mest, lokacijska koordinata, cena/h...","[0.8232184290885926, 0.7353307019832522, 0.701..."
5,Električna polnilnica,[],None,TN,[],[]
6,Število mest za invalide,[],None,TN,[],[]


Saved CSV: notebook_straza/results_eval/results_eval_json/eval_json_details_straza.csv


In [7]:
# Povzetek ključnih metrik
dm = open_world["decision_metrics"]  # to mora biti slovar!

if isinstance(dm, dict):
    print(f"Precision: {dm.get('precision', 0):.3f}")
    print(f"Recall:    {dm.get('recall', 0):.3f}")
    print(f"F1 score:  {dm.get('f1', 0):.3f}")
    print(f"Accuracy:  {dm.get('accuracy', 0):.3f}")
    print()
    print(f"TP={dm.get('TP')}, FP={dm.get('FP')}, FN={dm.get('FN')}, TN={dm.get('TN')}")
else:
    print("Napaka: open_world['decision_metrics'] je tipa", type(dm))
    print(dm)


Precision: 1.000
Recall:    1.000
F1 score:  1.000
Accuracy:  1.000

TP=5, FP=0, FN=0, TN=2


In [8]:
# Robust JSON export (handles Ellipsis, set/tuple, NaN, numpy/pandas types)
import json, datetime
from pathlib import Path

def to_jsonable(obj):
    # 1) direct fixes
    if obj is Ellipsis:
        return None
    # numpy scalars -> Python
    try:
        import numpy as np
        if isinstance(obj, (np.generic,)):
            return obj.item()
    except Exception:
        pass
    # pandas timestamps -> isoformat
    try:
        import pandas as pd
        if isinstance(obj, pd.Timestamp):
            return obj.isoformat()
        # pandas NA/NaT/NaN -> None
        try:
            if pd.isna(obj):
                return None
        except Exception:
            pass
    except Exception:
        pass

    # 2) recursive structures
    if isinstance(obj, dict):
        return {k: to_jsonable(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [to_jsonable(x) for x in obj]
    if isinstance(obj, tuple):
        return [to_jsonable(x) for x in obj]
    if isinstance(obj, set):
        # sort for stability; also sanitize each element
        return sorted([to_jsonable(x) for x in obj], key=lambda x: str(x))

    # 3) everything else stays as-is (str, int, float, bool, None)
    return obj

OUT_JSON = "notebook_straza/results_eval/eval_open_world_results.json"
Path(OUT_JSON).parent.mkdir(parents=True, exist_ok=True)

out = {
    "params": {
        "gold_json": GOLD_JSON,
        "preds_json": PREDS_JSON,
        "top_k": TOP_K,
        "threshold": THRESHOLD,
        "score_fields": SCORE_FIELDS,
        "timestamp": datetime.datetime.now().isoformat()
    },
    "open_world": open_world,
    "closed_world": closed_metrics
}

with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(to_jsonable(out), f, ensure_ascii=False, indent=2)

print("Saved JSON:", OUT_JSON)



Saved JSON: notebook_straza/results_eval/eval_open_world_results.json


/tmp/ipykernel_4039915/1653874963.py:23: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if pd.isna(obj):
